Adapted from https://github.com/NaysanSaran/pandas2postgresql/blob/master/notebooks/Psycopg2_Bulk_Insert_Speed_Benchmark.ipynb

In [1]:
#Load initial libraries
import os
from dotenv import load_dotenv, find_dotenv

In [2]:
#Get environment variables
load_dotenv(find_dotenv())
host2 = os.environ.get("DB_HOST")
database2 = os.environ.get("DB_NAME")
user2 = os.environ.get("DB_USER")
password2 = os.environ.get("DB_PASSWORD")

In [3]:
#connect to database
import psycopg2
import psycopg2.extras as extras

In [4]:
#Define a function to connect to the database. This was normally done with a one liner, and the example used a dict. 
#I ignored using the dict
def connect(host2, database2, user2, password2):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(host=host2, database=database2, user=user2, password=password2)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

conn = connect(host2, database2, user2, password2)

Connecting to the PostgreSQL database...
Connection successful


In [5]:
#connect to yahoo finance and load price dataframe
import yfinance as yf
import pandas as pd

ticker = yf.Ticker("AAPL")
df = ticker.history(period="max")
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1980-12-12,0.100600,0.101037,0.100600,0.100600,469033600,0.0,0.0
1980-12-15,0.095789,0.095789,0.095352,0.095352,175884800,0.0,0.0
1980-12-16,0.088790,0.088790,0.088353,0.088353,105728000,0.0,0.0
1980-12-17,0.090540,0.090977,0.090540,0.090540,86441600,0.0,0.0
1980-12-18,0.093165,0.093602,0.093165,0.093165,73449600,0.0,0.0


In [6]:
#Lots of cleanup needs to be done. 
#First is to change the Date "column" from an Index to a real column and format its contents.
df.reset_index(inplace=True)
df['Date'] = df['Date'].dt.strftime("%Y-%m-%d")
df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,1980-12-12,0.100600,0.101037,0.100600,0.100600,469033600,0.0,0.0
1,1980-12-15,0.095789,0.095789,0.095352,0.095352,175884800,0.0,0.0
2,1980-12-16,0.088790,0.088790,0.088353,0.088353,105728000,0.0,0.0
3,1980-12-17,0.090540,0.090977,0.090540,0.090540,86441600,0.0,0.0
4,1980-12-18,0.093165,0.093602,0.093165,0.093165,73449600,0.0,0.0


In [7]:
#The last two columns are not needed so they will be dropped.
df=df.drop(['Dividends', 'Stock Splits'], axis=1)
df.head()

,Date,Open,High,Low,Close,Volume
0,1980-12-12,0.100600,0.101037,0.100600,0.100600,469033600
1,1980-12-15,0.095789,0.095789,0.095352,0.095352,175884800
2,1980-12-16,0.088790,0.088790,0.088353,0.088353,105728000
3,1980-12-17,0.090540,0.090977,0.090540,0.090540,86441600
4,1980-12-18,0.093165,0.093602,0.093165,0.093165,73449600


In [8]:
#The columns will be renamed to match the case in the database
df=df.rename(columns={
    "Date" : "dt",
    "Open" : "open",
    "High" : "high",
    "Low" : "low",
    "Close" : "close",
    "Volume" : "volume"
    })
df.head()

,dt,open,high,low,close,volume
0,1980-12-12,0.100600,0.101037,0.100600,0.100600,469033600
1,1980-12-15,0.095789,0.095789,0.095352,0.095352,175884800
2,1980-12-16,0.088790,0.088790,0.088353,0.088353,105728000
3,1980-12-17,0.090540,0.090977,0.090540,0.090540,86441600
4,1980-12-18,0.093165,0.093602,0.093165,0.093165,73449600


In [9]:
#An additional column will be added to the table to signify the stock_id column in the database
df['stock_id'] = 1
df.head()

,dt,open,high,low,close,volume,stock_id
0,1980-12-12,0.100600,0.101037,0.100600,0.100600,469033600,1
1,1980-12-15,0.095789,0.095789,0.095352,0.095352,175884800,1
2,1980-12-16,0.088790,0.088790,0.088353,0.088353,105728000,1
3,1980-12-17,0.090540,0.090977,0.090540,0.090540,86441600,1
4,1980-12-18,0.093165,0.093602,0.093165,0.093165,73449600,1


In [10]:
#Define a helper function to use "execute_values"
def execute_values(conn, df, table):
    """
    Using psycopg2.extras.execute_values() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_values() done")
    cursor.close()

In [11]:
#Query for total count of symbols to get from Yahoo
cursor = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
cursor.execute("""
    SELECT COUNT(DISTINCT holding_id) from index_holdings
""")
rows = cursor.fetchall()
rows

[[3859]]

In [12]:
#Get all of the symbols and ids needed
cursor.execute("""
    SELECT * FROM tickers WHERE id IN(SELECT DISTINCT holding_id from index_holdings)
""")
rows = cursor.fetchall()
rows[:4]

[[3, 'FOLD', 'AMICUS THERAPEUTICS INC', 'BBG000PLF6N0', 'Common Stock'],
 [5, 'HVT', 'HAVERTY FURNITURE', 'BBG000FVBPJ6', 'Common Stock'],
 [6, 'LTRN', 'LANTERN PHARMA INC', 'BBG00GM6LK17', 'Common Stock'],
 [12, 'EBET', 'ESPORTS TECHNOLOGIES INC', 'BBG00ZLGHH52', 'Common Stock']]

In [13]:
#Try to run on all ~4k symbols
import time

for stock in rows:
    try:
        symbol = stock['symbol']
        ticker = yf.Ticker(symbol)
        df = ticker.history(period="max")
        df.reset_index(inplace=True)
        df['Date'] = df['Date'].dt.strftime("%Y-%m-%d")
        df = df.drop(['Dividends', 'Stock Splits'], axis=1)
        df=df.rename(columns={
            "Date" : "dt",
            "Open" : "open",
            "High" : "high",
            "Low" : "low",
            "Close" : "close",
            "Volume" : "volume"
        })
        df['stock_id'] = stock['id']
        execute_values(conn, df, 'stock_prices')
    except Exception as e:
        print(e)
    print(f'Succeeded in loading data for {symbol}.')
    time.sleep(1)
    
    
    
    

execute_values() done
Succeeded in loading data for FOLD.
execute_values() done
Succeeded in loading data for HVT.
execute_values() done
Succeeded in loading data for LTRN.
execute_values() done
Succeeded in loading data for EBET.
execute_values() done
Succeeded in loading data for SPRO.
execute_values() done
Succeeded in loading data for AGFS.
execute_values() done
Succeeded in loading data for CNSL.
execute_values() done
Succeeded in loading data for API.
execute_values() done
Succeeded in loading data for KDP.
execute_values() done
Succeeded in loading data for DCT.
execute_values() done
Succeeded in loading data for ALOT.
execute_values() done
Succeeded in loading data for SANM.
execute_values() done
Succeeded in loading data for ACR.
execute_values() done
Succeeded in loading data for FSLR.
execute_values() done
Succeeded in loading data for HBI.
execute_values() done
Succeeded in loading data for SAH.
execute_values() done
Succeeded in loading data for COLM.
execute_values() done

execute_values() done
Succeeded in loading data for CMP.
execute_values() done
Succeeded in loading data for TOWN.
execute_values() done
Succeeded in loading data for SYNA.
execute_values() done
Succeeded in loading data for EXTR.
execute_values() done
Succeeded in loading data for OTIC.
execute_values() done
Succeeded in loading data for SSB.
execute_values() done
Succeeded in loading data for BSGM.
execute_values() done
Succeeded in loading data for GOLF.
execute_values() done
Succeeded in loading data for SYF.
execute_values() done
Succeeded in loading data for VOR.
execute_values() done
Succeeded in loading data for BLD.
execute_values() done
Succeeded in loading data for EVBG.
execute_values() done
Succeeded in loading data for KFFB.
execute_values() done
Succeeded in loading data for XFOR.
execute_values() done
Succeeded in loading data for RPM.
execute_values() done
Succeeded in loading data for FIBK.
execute_values() done
Succeeded in loading data for ALX.
execute_values() done

execute_values() done
Succeeded in loading data for DAR.
execute_values() done
Succeeded in loading data for RBCAA.
execute_values() done
Succeeded in loading data for CADE.
execute_values() done
Succeeded in loading data for NVEE.
execute_values() done
Succeeded in loading data for TRIP.
execute_values() done
Succeeded in loading data for AXLA.
execute_values() done
Succeeded in loading data for VIAC.
execute_values() done
Succeeded in loading data for AIN.
execute_values() done
Succeeded in loading data for WRB.
execute_values() done
Succeeded in loading data for SNAP.
execute_values() done
Succeeded in loading data for FARM.
execute_values() done
Succeeded in loading data for XENT.
execute_values() done
Succeeded in loading data for MRAM.
Error: numeric field overflow
DETAIL:  A field with precision 6, scale 2 must round to an absolute value less than 10^4.

Succeeded in loading data for CVM.
execute_values() done
Succeeded in loading data for GMRE.
execute_values() done
Succeeded i

execute_values() done
Succeeded in loading data for FIS.
execute_values() done
Succeeded in loading data for ICE.
execute_values() done
Succeeded in loading data for RSG.
execute_values() done
Succeeded in loading data for TG.
execute_values() done
Succeeded in loading data for F.
execute_values() done
Succeeded in loading data for CRNC.
Error: numeric field overflow
DETAIL:  A field with precision 6, scale 2 must round to an absolute value less than 10^4.

Succeeded in loading data for CYCC.
execute_values() done
Succeeded in loading data for TACO.
execute_values() done
Succeeded in loading data for APPN.
execute_values() done
Succeeded in loading data for HOFV.
execute_values() done
Succeeded in loading data for NL.
execute_values() done
Succeeded in loading data for HYLN.
execute_values() done
Succeeded in loading data for BMTC.
execute_values() done
Succeeded in loading data for PLBY.
execute_values() done
Succeeded in loading data for FTK.
execute_values() done
Succeeded in loadin

execute_values() done
Succeeded in loading data for RPAY.
execute_values() done
Succeeded in loading data for SRPT.
execute_values() done
Succeeded in loading data for GLT.
execute_values() done
Succeeded in loading data for SAVA.
execute_values() done
Succeeded in loading data for AGNC.
execute_values() done
Succeeded in loading data for LII.
execute_values() done
Succeeded in loading data for TDY.
execute_values() done
Succeeded in loading data for CVAC.
execute_values() done
Succeeded in loading data for PRI.
execute_values() done
Succeeded in loading data for NCMI.
execute_values() done
Succeeded in loading data for TNK.
execute_values() done
Succeeded in loading data for ITCI.
execute_values() done
Succeeded in loading data for LILAK.
execute_values() done
Succeeded in loading data for HEPA.
execute_values() done
Succeeded in loading data for AM.
execute_values() done
Succeeded in loading data for ANAB.
execute_values() done
Succeeded in loading data for URGN.
execute_values() don

...

execute_values() done
Succeeded in loading data for AIRI.
execute_values() done
Succeeded in loading data for RHP.
execute_values() done
Succeeded in loading data for WWD.
execute_values() done
Succeeded in loading data for ESNT.
execute_values() done
Succeeded in loading data for PKG.
execute_values() done
Succeeded in loading data for LEA.
execute_values() done
Succeeded in loading data for HPP.
execute_values() done
Succeeded in loading data for LSXMK.
execute_values() done
Succeeded in loading data for FGBI.
execute_values() done
Succeeded in loading data for BWEN.
execute_values() done
Succeeded in loading data for TEN.
execute_values() done
Succeeded in loading data for LXRX.
execute_values() done
Succeeded in loading data for TIL.
execute_values() done
Succeeded in loading data for CLXT.
execute_values() done
Succeeded in loading data for EIGR.
execute_values() done
Succeeded in loading data for SMPL.
execute_values() done
Succeeded in loading data for AGM.
execute_values() done

execute_values() done
Succeeded in loading data for CNNE.
execute_values() done
Succeeded in loading data for GES.
execute_values() done
Succeeded in loading data for EPSN.
execute_values() done
Succeeded in loading data for GMBL.
execute_values() done
Succeeded in loading data for CYT.
execute_values() done
Succeeded in loading data for CPF.
execute_values() done
Succeeded in loading data for AMH.
execute_values() done
Succeeded in loading data for CTT.
execute_values() done
Succeeded in loading data for PBYI.
execute_values() done
Succeeded in loading data for FSR.
execute_values() done
Succeeded in loading data for NBSE.
execute_values() done
Succeeded in loading data for ADUS.
execute_values() done
Succeeded in loading data for SNCR.
execute_values() done
Succeeded in loading data for NOK.
execute_values() done
Succeeded in loading data for SABR.
execute_values() done
Succeeded in loading data for DRH.
execute_values() done
Succeeded in loading data for ALNA.
execute_values() done


execute_values() done
Succeeded in loading data for WNC.
execute_values() done
Succeeded in loading data for HES.
execute_values() done
Succeeded in loading data for MSVB.
Error: numeric field overflow
DETAIL:  A field with precision 6, scale 2 must round to an absolute value less than 10^4.

Succeeded in loading data for CUEN.
execute_values() done
Succeeded in loading data for CATO.
execute_values() done
Succeeded in loading data for NTB.
execute_values() done
Succeeded in loading data for UDR.
execute_values() done
Succeeded in loading data for DMTK.
execute_values() done
Succeeded in loading data for CNTY.
execute_values() done
Succeeded in loading data for LKFN.
execute_values() done
Succeeded in loading data for MCBC.
execute_values() done
Succeeded in loading data for AAT.
execute_values() done
Succeeded in loading data for HOTH.
execute_values() done
Succeeded in loading data for WMPN.
execute_values() done
Succeeded in loading data for BOX.
execute_values() done
Succeeded in l

execute_values() done
Succeeded in loading data for DEN.
execute_values() done
Succeeded in loading data for INVO.
execute_values() done
Succeeded in loading data for FIX.
execute_values() done
Succeeded in loading data for RYDBF.
execute_values() done
Succeeded in loading data for BKNG.
execute_values() done
Succeeded in loading data for VVV.
execute_values() done
Succeeded in loading data for TTD.
execute_values() done
Succeeded in loading data for MHK.
Error: numeric field overflow
DETAIL:  A field with precision 6, scale 2 must round to an absolute value less than 10^4.

Succeeded in loading data for PSTV.
execute_values() done
Succeeded in loading data for AOUT.
execute_values() done
Succeeded in loading data for NSSC.
execute_values() done
Succeeded in loading data for EOLS.
execute_values() done
Succeeded in loading data for STWD.
execute_values() done
Succeeded in loading data for KIRK.
execute_values() done
Succeeded in loading data for BKE.
execute_values() done
Succeeded in 

execute_values() done
Succeeded in loading data for OTLK.
execute_values() done
Succeeded in loading data for CRVS.
execute_values() done
Succeeded in loading data for DOV.
execute_values() done
Succeeded in loading data for JBHT.
execute_values() done
Succeeded in loading data for EQH.
execute_values() done
Succeeded in loading data for AEYE.
execute_values() done
Succeeded in loading data for BYND.
execute_values() done
Succeeded in loading data for LJPC.
execute_values() done
Succeeded in loading data for COHU.
execute_values() done
Succeeded in loading data for BMTX.
execute_values() done
Succeeded in loading data for TCON.
execute_values() done
Succeeded in loading data for ADVM.
execute_values() done
Succeeded in loading data for SEDG.
execute_values() done
Succeeded in loading data for ATRA.
execute_values() done
Succeeded in loading data for MSEX.
execute_values() done
Succeeded in loading data for LSF.
execute_values() done
Succeeded in loading data for GO.
execute_values() do